In [1]:
import audio_processing_helpers as aph
import song_joiner as joiner
import re
import os

In [7]:
import os
import re
from dataclasses import dataclass, field
from typing import Dict, List, Tuple
import audio_processing_helpers as aph

In [8]:
path = "/Users/fairviewmedia/Media/Music/MultiTracks/Man of Sorrows - At Easter/E"
tracker: Dict[Tuple[str, str, float], List[joiner.TrimmedMusicFile]] = {}
for item in os.listdir(path):
    mo = re.match(joiner.pattern, item)
    if mo:
        try:
            music_file = joiner.TrimmedMusicFile(os.path.join(path, item))
            key = (
                music_file.title,
                music_file.chord,
                music_file.trim_length,
            )
            if key not in tracker:
                tracker[key] = [music_file]
            else:
                tracker[key].append(music_file)
        except Exception as e:
            print(f"Error processing file {item}: {e}")
song_data: List[joiner.SongData] = []
for list_ in tracker.values():
    if len(list_) == 2:
        bare, bgv = sorted(list_, key=lambda x: x.has_bgv)
        song_data.append(joiner.SongData(bare=bare, bgv=bgv))


TypeError: attribute name must be string, not 'function'

In [10]:
bare, bgv

(TrimmedMusicFile(path='/Users/fairviewmedia/Media/Music/MultiTracks/Man of Sorrows - At Easter/E/Man of Sorrows-E-Trimmed_6.4205s.m4a', title='Man of Sorrows', chord='E', trim_length=6.4205, has_bgv=False),
 TrimmedMusicFile(path='/Users/fairviewmedia/Media/Music/MultiTracks/Man of Sorrows - At Easter/E/Man of Sorrows-E-with Background Vocals-Trimmed_6.4205s.m4a', title='Man of Sorrows', chord='E', trim_length=6.4205, has_bgv=True))

In [ ]:
topdir = "/Users/fairviewmedia/Media/Music/MultiTracks/"
for root, dirs, _ in os.walk(topdir):
    for dir_ in dirs:
        fulldir = os.path.join(root, dir_)
        pairs = joiner.find_pairs(fulldir)

        for p in pairs:
            if not os.path.exists(p.wav_path):
                p.make_split_wav()

In [ ]:
pattern = (
    r'^(?!\.)(?P<title>.+?)(?:-(?P<chord>[A-G](?:b|#)?)'
    r'(?:-with Background Vocals)?)'
    r'?-Trimmed_(?P<trim_length>\d+(?:\.\d+)?)s\.m4a$'
 )

sample = dict(title='hey', chord='C', '')

pattern.format_map()

In [17]:
import subprocess
import numpy as np
from scipy.signal import correlate
from numpy.typing import ArrayLike

def load_audio_array(input_file, sample_rate=16000) -> ArrayLike:
    # ffmpeg command to output raw PCM 16-bit little endian audio
    cmd = [
        'ffmpeg', '-i', input_file,
        '-f', 's16le',  # PCM 16-bit little endian
        '-ac', '1',     # mono
        '-ar', str(sample_rate),  # sampling rate
        '-loglevel', 'quiet',
        '-'
    ]
    process = subprocess.run(cmd, stdout=subprocess.PIPE, check=True)
    audio_bytes = process.stdout
    audio_array = np.frombuffer(audio_bytes, np.int16).astype(np.float32)
    audio_array /= np.max(np.abs(audio_array))  # normalize
    return audio_array

def find_offset_in_memory(fileA, fileB, sample_rate=16000):
    dataA = load_audio_array(fileA, sample_rate)
    dataB = load_audio_array(fileB, sample_rate)

    correlation = correlate(dataB, dataA, mode='full')
    lag = correlation.argmax() - (len(dataA) - 1)
    return lag / sample_rate

# offset = find_offset_in_memory('fileA.mp3', 'fileB.mp4')
# print(f"Offset: {offset:.3f} seconds")

import os
dir_ = "/Users/fairviewmedia/Media/Music/MultiTracks/Who You Say I Am - There Is More"
f1 = os.path.join(dir_, "Who You Say I Am (Lyric Video) - Hillsong Worship [IcC1Bp13n_4].m4a")
f2 = os.path.join(dir_, "G", "Who You Say I Am-G-with Background Vocals.m4a")


In [15]:
load_audio_array(f1)

array([0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 9.1552734e-05,
       6.1035156e-05, 0.0000000e+00], shape=(5264800,), dtype=float32)

In [11]:
find_offset_in_memory(f1, f2)

np.float64(2.852375)

In [ ]:
# Pad Worthy by 1.4970625s

In [ ]:
5.1560625